<a href="https://colab.research.google.com/github/Paradorn657/CNNs/blob/master/text%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install attacut

In [0]:
from attacut import tokenize

In [0]:
Xtrain = ["อ่อนแอก็แพ้ไป คนไหวเขาจะยืน",
          "หวยออกต้นเดือน",
          "สวัสดีครับ คุณผู้ชม",
          "เบื่อความร้อนต้องการความรัก",
          "ผู้ไม่มีแผลเป็นคือ ผู้ไม่มีประสบการณ์",
          "ความลำบากที่เกินทน จะหลอมคนให้ทนทาน",
          "อวกาศ เป็นที่มืด"
          ]
Ytrain = ["คำคม",
          "ไม่ใช่คำคม",
          "ไม่ใช่คำคม",
          "คำคม",
          "คำคม",
          "คำคม",
          "ไม่ใช่คำคม"
          ]
Xtest = ["เมื่อไม่มีสิ่งที่ชอบ ก็ต้องชอบสิ่งที่มี","ภูเขา สวยดี","ผมรักคุณ"]

In [0]:
import gdown
import tensorflow as tf
from tensorflow import keras
import numpy as np
import re

In [33]:
gdown.download('https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download', 'th.zip', quiet=False) 

Downloading...
From: https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download
To: /content/th.zip
122MB [00:01, 99.0MB/s]


'th.zip'

In [34]:
!unzip th.zip

Archive:  th.zip
replace th.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: th.bin                  
  inflating: th.tsv                  


In [35]:
with open('th.tsv') as f:
  vocab = []
  W = None
  vec = ''
  for line in f.readlines():
    temp = line.split('\t')
    if len(temp) == 3:
      vocab.append(temp[1])
      if int(temp[0]) % 1000 == 0:
        print('\r' + temp[0], end='')
      wordvector = np.fromstring(re.sub('\s+', ' ', vec)[1:-1], sep=' ', dtype=np.float32)
      if len(wordvector) > 0:
        if W is None:
          W = wordvector.copy()
        else:
          W = np.vstack((W, wordvector))
      vec = temp[-1]
    elif len(temp) == 1:
      vec += temp[-1]
  W = np.vstack((W, wordvector))
print('\n')
print(len(vocab))
print(W.shape)

0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  # This is added back by InteractiveShellApp.init_path()


30000

30225
(30225, 300)


In [0]:
def sent2idx(x):
  xidx = []
  for w in tokenize(x):
    if w in vocab:
      xidx.append(vocab.index(w))
  return np.array(xidx)

In [37]:
tokenize(Xtrain[2])

['สวัสดี', 'ครับ', ' ', 'คุณ', 'ผู้', 'ชม']

In [38]:
Xtrain_idx = []
maxlen = 0
for x in Xtrain:
  Xtrain_idx.append(sent2idx(x))
  if len(Xtrain_idx[-1]) > maxlen:
    maxlen = len(Xtrain_idx[-1])
  print(len(Xtrain_idx[-1]))

9
4
5
6
10
10
4


In [39]:
Xtrain_idx

[array([3366,   27,  518,   17,   30, 1677,   52,   13, 1562]),
 array([9452,   39,  297,  182]),
 array([4740,  346,  317,   22,  814]),
 array([4343,   16,  307,  465,   16,  164]),
 array([  22,   20,    7, 2851,    2,   46,   22,   20,    7, 1831]),
 array([  16, 2863,    0,  638, 2564,   13, 4934,   30,   18, 5715]),
 array([1364,    2,    0, 1980])]

In [40]:
len(vocab)

30225

In [41]:
vocab = [''] + vocab
print(len(vocab))

30226


In [42]:
W.shape

(30225, 300)

In [0]:
W = np.vstack((np.random.rand(*W[0].shape), W))

In [0]:
W[1]

In [45]:
vocab[0]

''

In [0]:
for i,x in enumerate(Xtrain_idx):
    if len(x) < maxlen:
      Xtrain_idx[i] = np.hstack((x,np.zeros(maxlen-len(x))))
Xtrain_idx = np.array(Xtrain_idx)

In [47]:
Xtrain_idx.shape

(7, 10)

In [0]:
model = keras.Sequential()
model.add(keras.layers.Embedding(W.shape[0], W.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(4)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2,activation="softmax"))

model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam())

In [0]:
model.get_layer('embed').set_weights([W])
model.get_layer('embed').trainable = False

In [0]:
model.fit(Xtrain_idx, np.array([1,0,0,1,1,1,0]), epochs=100)

In [60]:
model.predict(Xtrain_idx).argmax(axis=1)

array([1, 0, 0, 1, 1, 1, 0])

In [0]:
Xtest_idx = []
for x in Xtest:
  Xtest_idx.append(sent2idx(x))
  if len(Xtest_idx[-1]) > maxlen:
    Xtest_idx[-1] =  Xtest_idx[-1][:maxlen]
  else:
    Xtest_idx[-1] = np.hstack((Xtest_idx[-1], np.zeros(maxlen-len(Xtest_idx[-1]))))
Xtest_idx = np.array(Xtest_idx)

In [70]:
Xtest_idx

array([[3.400e+01, 2.100e+01, 8.000e+00, 3.050e+02, 1.000e+00, 5.210e+02,
        2.800e+01, 9.900e+01, 5.210e+02, 3.050e+02],
       [1.426e+03, 1.037e+03, 7.800e+01, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00]])

In [71]:
model.predict(Xtest_idx).argmax(axis=1)

array([1, 0, 0])